<a href="https://colab.research.google.com/github/catch-twenty2/AstroChart_Analysis/blob/main/AstroChartAnalysis3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#First time before running the code you need to install these libraries
!pip install flatlib pyswisseph timezonefinder pytz ipywidgets geopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.6/521.6 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.1 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for pyswisseph: filename=pyswisseph-2.8.0.post1-cp310-cp310-linux_x86_64.whl size=838863 sha256=4f1d00bcc5a41f26a67afe2b07bd517e258ae5ebcc23b341cc085923f3417452
  Stored in directory: /root/.cache/pip/wheels/36/dd/88/1294ee56efe50a75c487567ec59244f4eaf7f3956fa0dcefe6
Successfully built pyswisseph


In [3]:
#then re-run this code anytime you want to adjust the input
import ipywidgets as widgets
from IPython.display import display, clear_output
from flatlib import const
from flatlib.chart import Chart
from flatlib.geopos import GeoPos
from flatlib.datetime import Datetime
from datetime import datetime, timedelta
from timezonefinder import TimezoneFinder
import pytz
import swisseph as swe
import math
from geopy.geocoders import Nominatim

# Define rulership for each zodiac sign
planetary_rulers = {
    'Aries': 'Mars', 'Taurus': 'Venus', 'Gemini': 'Mercury', 'Cancer': 'Moon',
    'Leo': 'Sun', 'Virgo': 'Mercury', 'Libra': 'Venus', 'Scorpio': 'Pluto',
    'Sagittarius': 'Jupiter', 'Capricorn': 'Saturn', 'Aquarius': 'Uranus',
    'Pisces': 'Neptune'
}

# Helper function to determine the sign of a planet's longitude
def get_sign(longitude):
    signs = ['Aries', 'Taurus', 'Gemini', 'Cancer', 'Leo', 'Virgo',
             'Libra', 'Scorpio', 'Sagittarius', 'Capricorn', 'Aquarius', 'Pisces']
    return signs[int(longitude / 30)]

# Helper function to find the planetary ruler of a sign
def get_ruler(sign):
    return planetary_rulers[sign]

# Helper function to determine the house of a planet based on longitude using house cusps
def get_house_for_swisseph_planets(longitude, house_cusps):
    if longitude >= 360:
        longitude -= 360
    for i in range(1, 13):  # 1 to 12 houses
        cusp_start = house_cusps[i - 1]
        cusp_end = house_cusps[i % 12]
        if cusp_end < cusp_start:
            if longitude >= cusp_start or longitude < cusp_end:
                return i
        else:
            if cusp_start <= longitude < cusp_end:
                return i
    return 12  # Default to 12th house if not found

# Calculate the difference between two angles (longitudes) in a circular system (360 degrees)
def angular_distance(lon1, lon2):
    diff = abs(lon1 - lon2)
    if diff > 180:
        diff = 360 - diff
    return diff

# Helper function to check for aspects
def get_aspect(angle):
    aspects = {
        'Conjunction': 0,
        'Opposition': 180,
        'Trine': 120,
        'Square': 90,
        'Sextile': 60,
        'Quincunx': 150
    }
    orbs = {
        'Conjunction': 8,
        'Opposition': 8,
        'Trine': 8,
        'Square': 8,
        'Sextile': 6,
        'Quincunx': 3
    }

    for aspect, base_angle in aspects.items():
        if abs(angle - base_angle) <= orbs[aspect]:
            return aspect, abs(angle - base_angle)
    return None, None

# Function to calculate and display natal chart aspects
def display_natal_aspects(natal_chart_positions):
    print("\nAspects:")
    planet_names = list(natal_chart_positions.keys())

    for i in range(len(planet_names)):
        for j in range(i + 1, len(planet_names)):
            planet1 = planet_names[i]
            planet2 = planet_names[j]
            lon1 = natal_chart_positions[planet1]['longitude']
            lon2 = natal_chart_positions[planet2]['longitude']
            angle = angular_distance(lon1, lon2)
            aspect, orb = get_aspect(angle)
            if aspect:
                print(f"{planet1} {aspect} {planet2} (orb: {orb:.2f})")

# Function to calculate and display transit aspects
def display_transit_aspects(natal_chart_positions, transit_positions):
    print("\nTransit Aspects:")
    for transit_planet, transit_data in transit_positions.items():
        for natal_planet, natal_data in natal_chart_positions.items():
            transit_longitude = transit_data['longitude']
            natal_longitude = natal_data['longitude']
            angle = angular_distance(transit_longitude, natal_longitude)
            aspect, orb = get_aspect(angle)
            if aspect:
                print(f"Transit {transit_planet} {aspect} Natal {natal_planet} (orb: {orb:.2f})")

# Function to get city and country from latitude and longitude
def get_city_country(latitude, longitude):
    geolocator = Nominatim(user_agent="astro_app")
    location = geolocator.reverse((latitude, longitude), exactly_one=True)
    if location and location.raw.get('address'):
        address = location.raw['address']
        city = address.get('city', address.get('town', address.get('village', 'Unknown')))
        country = address.get('country', 'Unknown')
        return city, country
    return "Unknown", "Unknown"

# Function to calculate UTC offset based on birth date and location
def calculate_utc_offset(year, month, day, hour, minute, latitude, longitude):
    tf = TimezoneFinder()
    timezone_str = tf.timezone_at(lat=latitude, lng=longitude)

    if timezone_str is None:
        raise ValueError("Could not determine the timezone for the given location.")

    local_timezone = pytz.timezone(timezone_str)
    naive_local_time = datetime(year, month, day, hour, minute)
    local_time = local_timezone.localize(naive_local_time, is_dst=None)

    utc_offset_timedelta = local_time.utcoffset()
    utc_offset_hours = utc_offset_timedelta.total_seconds() / 3600

    return utc_offset_hours

# Define the signs and their corresponding elements, modes, and polarities
sign_elements = {
    'Aries': 'Fire', 'Taurus': 'Earth', 'Gemini': 'Air', 'Cancer': 'Water',
    'Leo': 'Fire', 'Virgo': 'Earth', 'Libra': 'Air', 'Scorpio': 'Water',
    'Sagittarius': 'Fire', 'Capricorn': 'Earth', 'Aquarius': 'Air', 'Pisces': 'Water'
}

sign_modes = {
    'Aries': 'Cardinal', 'Taurus': 'Fixed', 'Gemini': 'Mutable', 'Cancer': 'Cardinal',
    'Leo': 'Fixed', 'Virgo': 'Mutable', 'Libra': 'Cardinal', 'Scorpio': 'Fixed',
    'Sagittarius': 'Mutable', 'Capricorn': 'Cardinal', 'Aquarius': 'Fixed', 'Pisces': 'Mutable'
}

sign_polarities = {
    'Aries': 'Masculine', 'Taurus': 'Feminine', 'Gemini': 'Masculine', 'Cancer': 'Feminine',
    'Leo': 'Masculine', 'Virgo': 'Feminine', 'Libra': 'Masculine', 'Scorpio': 'Feminine',
    'Sagittarius': 'Masculine', 'Capricorn': 'Feminine', 'Aquarius': 'Masculine', 'Pisces': 'Feminine'
}

# Function to count the elements, modes, and polarities
def count_elements_modes_polarities(natal_chart_positions):
    element_count = {'Fire': 0, 'Earth': 0, 'Air': 0, 'Water': 0}
    mode_count = {'Cardinal': 0, 'Fixed': 0, 'Mutable': 0}
    polarity_count = {'Masculine': 0, 'Feminine': 0}

    for planet, data in natal_chart_positions.items():
        sign = data['sign']
        element = sign_elements[sign]
        mode = sign_modes[sign]
        polarity = sign_polarities[sign]

        element_count[element] += 1
        mode_count[mode] += 1
        polarity_count[polarity] += 1

    return element_count, mode_count, polarity_count

# Function to count the hemisphere and quadrant balance
def count_hemisphere_quadrant_balance(natal_chart_positions):
    hemisphere_count = {'Eastern': 0, 'Western': 0, 'Northern': 0, 'Southern': 0}
    quadrant_count = {'First': 0, 'Second': 0, 'Third': 0, 'Fourth': 0}

    for planet, data in natal_chart_positions.items():
        house = int(data['house'])

        # Count hemisphere balance
        if house in [10, 11, 12, 1, 2, 3]:
            hemisphere_count['Eastern'] += 1
        else:
            hemisphere_count['Western'] += 1

        if house in [1, 2, 3, 4, 5, 6]:
            hemisphere_count['Northern'] += 1
        else:
            hemisphere_count['Southern'] += 1

        # Count quadrant balance
        if house in [1, 2, 3]:
            quadrant_count['First'] += 1
        elif house in [4, 5, 6]:
            quadrant_count['Second'] += 1
        elif house in [7, 8, 9]:
            quadrant_count['Third'] += 1
        elif house in [10, 11, 12]:
            quadrant_count['Fourth'] += 1

    return hemisphere_count, quadrant_count

# Function to add the Lunar Nodes, Chiron, and Asteroids to the natal chart positions
def add_lunar_nodes_chiron_and_asteroids(natal_chart_positions, jd, house_cusps):
    # Calculate North Node (True Node)
    north_node = swe.calc_ut(jd, swe.TRUE_NODE)[0]
    natal_chart_positions['North Node'] = {
        'longitude': north_node[0],
        'latitude': north_node[1],  # Add latitude
        'sign': get_sign(north_node[0]),
        'house': get_house_for_swisseph_planets(north_node[0], house_cusps),
        'ruler': get_ruler(get_sign(north_node[0]))  # Add ruler
    }

    # South Node is directly opposite to the North Node
    south_node_longitude = (north_node[0] + 180) % 360
    natal_chart_positions['South Node'] = {
        'longitude': south_node_longitude,
        'latitude': -north_node[1],  # Latitude is opposite for South Node
        'sign': get_sign(south_node_longitude),
        'house': get_house_for_swisseph_planets(south_node_longitude, house_cusps),
        'ruler': get_ruler(get_sign(south_node_longitude))  # Add ruler
    }

    # Calculate Chiron
    chiron = swe.calc_ut(jd, swe.CHIRON)[0]
    natal_chart_positions['Chiron'] = {
        'longitude': chiron[0],
        'latitude': chiron[1],  # Add latitude
        'sign': get_sign(chiron[0]),
        'house': get_house_for_swisseph_planets(chiron[0], house_cusps),
        'ruler': get_ruler(get_sign(chiron[0]))  # Add ruler
    }

    # Add asteroids (Ceres, Pallas, Juno, Vesta)
    asteroids = {
        'Ceres': swe.CERES,
        'Pallas': swe.PALLAS,
        'Juno': swe.JUNO,
        'Vesta': swe.VESTA
    }
    for asteroid_name, asteroid_id in asteroids.items():
        position = swe.calc_ut(jd, asteroid_id)[0]
        natal_chart_positions[asteroid_name] = {
            'longitude': position[0],
            'latitude': position[1],  # Add Latitude
            'sign': get_sign(position[0]),
            'house': get_house_for_swisseph_planets(position[0], house_cusps),
            'ruler': get_ruler(get_sign(position[0]))  # Add Ruler
        }

# Calculate declinations and identify parallels/contraparallels
def calculate_declinations_and_parallels(natal_chart_positions, jd):
    declinations = {}
    parallels = []
    contraparallels = []

    # Planets that support declinations
    planet_ids_with_declination = [
        ('Sun', swe.SUN), ('Moon', swe.MOON), ('Mercury', swe.MERCURY),
        ('Venus', swe.VENUS), ('Mars', swe.MARS), ('Jupiter', swe.JUPITER),
        ('Saturn', swe.SATURN), ('Uranus', swe.URANUS), ('Neptune', swe.NEPTUNE), ('Pluto', swe.PLUTO)
    ]

    # Calculate declinations for each planet
    for planet_name, planet_id in planet_ids_with_declination:
        planet_data = swe.calc_ut(jd, planet_id)
        if len(planet_data) > 2:
            declinations[planet_name] = planet_data[2]  # Declination is the third value in the result (degrees)

    # Compare declinations for parallels/contraparallels
    planet_names = list(declinations.keys())

    for i in range(len(planet_names)):
        for j in range(i + 1, len(planet_names)):
            planet1 = planet_names[i]
            planet2 = planet_names[j]
            decl1 = declinations.get(planet1, None)
            decl2 = declinations.get(planet2, None)

            if decl1 is not None and decl2 is not None:
                # Check for parallels (same hemisphere, close declinations within 1 degree)
                if abs(decl1 - decl2) <= 1 and (decl1 * decl2) > 0:  # Both positive or negative
                    parallels.append((planet1, planet2))

                # Check for contraparallels (opposite hemispheres, close declinations within 1 degree)
                if abs(decl1 - decl2) <= 1 and (decl1 * decl2) < 0:  # One positive, one negative
                    contraparallels.append((planet1, planet2))

    return declinations, parallels, contraparallels

# Function to display declinations and parallels/contraparallels
def display_declinations_and_parallels(declinations, parallels, contraparallels):
    print("\nDeclinations:")
    for planet, decl in declinations.items():
        print(f"{planet}: {decl:.2f}°")

    if parallels:
        print("\nParallels:")
        for parallel in parallels:
            print(f"Parallel between {parallel[0]} and {parallel[1]}")
    else:
        print("\nNo Parallels found.")

    if contraparallels:
        print("\nContraparallels:")
        for contraparallel in contraparallels:
            print(f"Contraparallel between {contraparallel[0]} and {contraparallel[1]}")
    else:
        print("\nNo Contraparallels found.")

# List of fixed stars to include in the analysis
fixed_stars = ['Regulus', 'Sirius', 'Aldebaran', 'Spica', 'Antares', 'Fomalhaut', 'Vega']

# Calculate fixed star positions and conjunctions
def calculate_fixed_star_conjunctions(natal_chart_positions, jd):
    star_conjunctions = []
    for star_name in fixed_stars:
        try:
            # Get the longitude of the fixed star using Swiss Ephemeris
            star_data = swe.fixstar_ut(star_name, jd)
            star_longitude = star_data[0][0]  # First item in the result is the longitude

            # Check each planet in the natal chart for conjunction with the fixed star
            for planet, planet_data in natal_chart_positions.items():
                planet_longitude = planet_data['longitude']

                # Check for conjunction within 1 degree orb
                if abs(angular_distance(planet_longitude, star_longitude)) <= 1:
                    star_conjunctions.append(f"{planet} is conjunct {star_name} (Longitude: {star_longitude:.2f}°)")

        except Exception as e:
            print(f"Error fetching data for star {star_name}: {e}")

    return star_conjunctions

# Function to display fixed star conjunctions
def display_fixed_star_conjunctions(fixed_star_conjunctions):
    print("\nFixed Star Conjunctions:")
    if fixed_star_conjunctions:
        for conjunction in fixed_star_conjunctions:
            print(conjunction)
    else:
        print("No significant fixed star conjunctions found.")

# Create widgets for user input with default values
user_name_input = widgets.Text(value='Neo', description="User Name:")  # Prefilled with "Neo"
year_input = widgets.IntText(value=1990, description="Birth Year:")
month_input = widgets.IntText(value=12, description="Birth Month:")
day_input = widgets.IntText(value=9, description="Birth Day:")
time_of_birth_input = widgets.Text(value="08:40", description="Birth Time (HH:MM):")
latitude_input = widgets.FloatText(value=-13.5319, description="Latitude:")
longitude_input = widgets.FloatText(value=-71.9675, description="Longitude:")

# New widget for target date input at the bottom, prefilled
target_year_input = widgets.IntText(value=2024, description="Target Year:")  # Prefilled
target_month_input = widgets.IntText(value=12, description="Target Month:")   # Prefilled
target_day_input = widgets.IntText(value=31, description="Target Day:")       # Prefilled

button = widgets.Button(description="Calculate Chart")
output = widgets.Output()

# Function to handle the button click and perform the calculation
def on_button_click(b):
    with output:
        clear_output()

        natal_chart_positions = {}

        try:
            # Capture input from widgets
            user_name = user_name_input.value
            year = year_input.value
            month = month_input.value
            day = day_input.value
            time_of_birth = time_of_birth_input.value + ":00"
            latitude = latitude_input.value
            longitude = longitude_input.value

            # Capture the target date from the bottom inputs
            target_year = target_year_input.value
            target_month = target_month_input.value
            target_day = target_day_input.value

            # Calculate the UTC offset automatically
            hour = int(time_of_birth[:2])
            minute = int(time_of_birth[3:5])
            utc_offset = calculate_utc_offset(year, month, day, hour, minute, latitude, longitude)

            # Convert local time to UTC
            local_birth_time = datetime(year, month, day, hour, minute)
            utc_birth_time = local_birth_time - timedelta(hours=utc_offset)

            # Get city and country from latitude and longitude
            city, country = get_city_country(latitude, longitude)

            # Format the UTC time for flatlib
            utc_birth_date_str = utc_birth_time.strftime("%Y/%m/%d")
            utc_birth_time_str = utc_birth_time.strftime("%H:%M:%S")

            # Create Datetime and GeoPos objects (for flatlib, if needed)
            birth_datetime = Datetime(utc_birth_date_str, utc_birth_time_str)
            birth_location = GeoPos(latitude, longitude)

            # Calculate the Julian Day
            jd = swe.julday(utc_birth_time.year, utc_birth_time.month, utc_birth_time.day,
                            utc_birth_time.hour + utc_birth_time.minute / 60.0)
            swe.set_topo(longitude, latitude, 0)  # Set location for swisseph

            # Calculate house cusps first
            house_cusps, ascmc = swe.houses(jd, latitude, longitude, b'P')

            # Calculate positions for all planets using swisseph
            traditional_planets = {
                'Sun': swe.SUN,
                'Moon': swe.MOON,
                'Mercury': swe.MERCURY,
                'Venus': swe.VENUS,
                'Mars': swe.MARS,
                'Jupiter': swe.JUPITER,
                'Saturn': swe.SATURN
            }

            for planet_name, planet_id in traditional_planets.items():
                position = swe.calc_ut(jd, planet_id)[0]
                longitude = position[0]
                latitude = position[1]
                natal_chart_positions[planet_name] = {
                    'longitude': longitude,
                    'latitude': latitude,
                    'sign': get_sign(longitude),
                    'house': get_house_for_swisseph_planets(longitude, house_cusps),
                    'ruler': get_ruler(get_sign(longitude))
                }

            modern_planets = {
                'Uranus': swe.URANUS,
                'Neptune': swe.NEPTUNE,
                'Pluto': swe.PLUTO
            }

            for planet_name, planet_id in modern_planets.items():
                position = swe.calc_ut(jd, planet_id)[0]
                longitude = position[0]
                latitude = position[1]
                natal_chart_positions[planet_name] = {
                    'longitude': longitude,
                    'latitude': latitude,
                    'sign': get_sign(longitude),
                    'house': get_house_for_swisseph_planets(longitude, house_cusps),
                    'ruler': get_ruler(get_sign(longitude))
                }

            # Calculate transit positions for the target date
            target_jd = swe.julday(target_year, target_month, target_day, 0)
            transit_positions = {}

            all_planets = {**traditional_planets, **modern_planets}

            for planet_name, planet_id in all_planets.items():
                position = swe.calc_ut(target_jd, planet_id)[0]
                longitude = position[0]
                transit_positions[planet_name] = {'longitude': longitude, 'sign': get_sign(longitude)}

            # Add Lunar Nodes, Chiron, and Asteroids to the natal chart positions
            add_lunar_nodes_chiron_and_asteroids(natal_chart_positions, jd, house_cusps)

            # Calculate retrogrades
            retrograde_planets = []
            for planet_name in natal_chart_positions:
                planet_id = getattr(swe, planet_name.upper(), None)
                if planet_id:
                    retrograde = swe.calc_ut(jd, planet_id)[0][3] < 0
                    natal_chart_positions[planet_name]['retrograde'] = retrograde
                    if retrograde:
                        retrograde_planets.append(planet_name)

            # Output Astrological Data
            print(f"\nDear Assistant, act as a professional Astrologer. Below is detailed astrological data for interpretation.\n")
            print(f"**Astrological Data:**")
            print(f"Name: {user_name}")
            print(f"Place of Birth: {city}, {country}")
            print(f"Date of Birth: {year}-{month:02d}-{day:02d}")
            print(f"Time of Birth: {time_of_birth} (UTC Offset: {utc_offset:+.2f})")
            print(f"UTC Time of Birth: {utc_birth_time.strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"Target Date: {target_year}-{target_month:02d}-{target_day:02d}\n")

            # Output Natal Chart Positions with Lon/Lat
            print("\nNatal Chart Positions:")
            for planet, data in natal_chart_positions.items():
                print(f"{planet}: {data['sign']} (Ruler: {data['ruler']}) in House {data['house']} at Lon: {data['longitude']:.2f}°, Lat: {data['latitude']:.2f}°")

            # Calculate and display Ascendant and Midheaven
            ascendant = ascmc[0]
            mc = ascmc[1]
            print(f"\nAscendant: {get_sign(ascendant)} ({ascendant:.2f}°)")
            print(f"Midheaven: {get_sign(mc)} ({mc:.2f}°)")

            # Calculate and display Element, Mode, Polarity
            element_count, mode_count, polarity_count = count_elements_modes_polarities(natal_chart_positions)
            hemisphere_count, quadrant_count = count_hemisphere_quadrant_balance(natal_chart_positions)

            print("\nElement Distribution:")
            for element, count in element_count.items():
                print(f"{element}: {count}")

            print("\nMode Distribution:")
            for mode, count in mode_count.items():
                print(f"{mode}: {count}")

            print("\nPolarity Distribution:")
            for polarity, count in polarity_count.items():
                print(f"{polarity}: {count}")

            print("\nHemisphere Balance:")
            for hemisphere, count in hemisphere_count.items():
                print(f"{hemisphere}: {count}")

            print("\nQuadrant Balance:")
            for quadrant, count in quadrant_count.items():
                print(f"{quadrant}: {count}")

            print("\nRetrograde Planets:")
            if retrograde_planets:
                for planet in retrograde_planets:
                    print(f"{planet} is retrograde.")
            else:
                print("No planets are retrograde.")

            # Calculate and display Declinations and Parallels
            declinations, parallels, contraparallels = calculate_declinations_and_parallels(natal_chart_positions, jd)
            display_declinations_and_parallels(declinations, parallels, contraparallels)

            # Calculate and display Fixed Star Conjunctions
            fixed_star_conjunctions = calculate_fixed_star_conjunctions(natal_chart_positions, jd)
            display_fixed_star_conjunctions(fixed_star_conjunctions)

            # Display Natal and Transit Aspects
            display_natal_aspects(natal_chart_positions)
            display_transit_aspects(natal_chart_positions, transit_positions)

            print(
                "\n**Interpretation Guidelines:**\n"
                "1. **Natal Chart Analysis:**\n"
                "   - Examine the individual's personality traits, strengths, and potential challenges based on the Sun, Moon, and Ascendant.\n"
                "   - Highlight important aspects between personal planets and outer planets to identify areas of potential growth or difficulty.\n\n"
                "2. **Transit Analysis:**\n"
                "   - Analyze how transiting planets interact with the natal chart, emphasizing long-term influences from outer planets.\n"
                "   - Focus on aspects between natal planets and transiting planets that suggest turning points or opportunities.\n\n"
                "3. **Key Themes and Life Cycles:**\n"
                "   - Summarize life themes and cycles emerging from both the natal and transit charts.\n"
                "   - Describe how these influences are likely to manifest in areas like career, relationships, or personal growth.\n\n"
                "4. **Practical Guidance and Recommendations:**\n"
                "   - Provide actionable advice on how to navigate any challenging transits.\n"
                "   - Highlight opportunities for growth or positive change, and suggest areas for further exploration.\n\n"
                "Finally, if there are any specific challenges or problematic aspects in the chart, recommend follow-up questions or next steps for further exploration.\n\nThank you!")

        except Exception as e:
            print(f"An error occurred: {e}")

# Display input fields and button
display(user_name_input, year_input, month_input, day_input, time_of_birth_input, latitude_input, longitude_input,
        target_year_input, target_month_input, target_day_input, button, output)

# Set the button click handler
button.on_click(on_button_click)

Text(value='Neo', description='User Name:')

IntText(value=1990, description='Birth Year:')

IntText(value=12, description='Birth Month:')

IntText(value=9, description='Birth Day:')

Text(value='08:40', description='Birth Time (HH:MM):')

...

IntText(value=2024, description='Target Year:')

IntText(value=12, description='Target Month:')

IntText(value=31, description='Target Day:')

Button(description='Calculate Chart', style=ButtonStyle())

Output()